<a href=\\\"https://colab.research.google.com/github/ulissigroup/amptorch/blob/active_learning/amptorch/active_learning/examples/neb_al_example.ipynb\\\" target=\\\"_parent\\\"><img src=\\\"https://colab.research.google.com/assets/colab-badge.svg\\\" alt=\\\"Open In Colab\\\"/></a>

In [0]:
!pip install -q git+https://github.com/ulissigroup/amptorch.git@active_learning
!pip install -q git+https://github.com/mshuaibii/SIMPLE-NN.git
!pip install -q amp-atomistics

In [0]:
import copy
import os
import numpy as np
import random

import torch

from amptorch.active_learning.learner import AtomisticActiveLearner
from amptorch.active_learning.query_methods import random_query, max_uncertainty
from amptorch.model import CustomMSELoss

import ase
from ase.calculators.calculator import Calculator
from ase.calculators.emt import EMT
from ase.calculators.singlepoint import SinglePointCalculator as sp
from ase.optimize import QuasiNewton
from ase.neb import NEB, SingleCalculatorNEB
from ase.optimize import BFGS, FIRE, BFGSLineSearch
from ase import Atoms, Atom, units
from ase.build import molecule
from ase.visualize import view
from ase.io import read, write
from ase.build import fcc100, add_adsorbate,bcc100,fcc111
from ase.constraints import FixAtoms
from ase.neb import NEBTools

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline 


In [0]:
def attach_sp_calc(image):
  "Converts image calculator to single point calculator to avoid double calculation."
  sample_energy = image.get_potential_energy(apply_constraint=False)
  sample_forces = image.get_forces(apply_constraint=False)
  image.set_calculator(
      sp(atoms=image, energy=sample_energy, forces=sample_forces)
  )
  return image

class CounterCalc(Calculator):
  "ASE calculator to track the number of force calls the parent calculator has performed"
  implemented_properties = ["energy", "forces"]

  def __init__(self, calc):
    """Parameters
    --------------
      calc: object. Parent calculator to track force calls."""
    super().__init__()
    self.calc = calc
    self.force_calls = 0

  def calculate(self, atoms, properties, system_changes):
    super().calculate(atoms, properties, system_changes)
    calc = copy.deepcopy(self.calc)
    self.results["energy"] = calc.get_potential_energy(atoms)
    self.results["forces"] = calc.get_forces(atoms)
    self.force_calls += 1


class NEBcalc:
    def __init__(self, starting_images, ml2relax=True, intermediate_samples=3):
      """
      Computes a NEB given an initial and final image.

      Parameters
      ----------
      starting_images: list. Initial and final images to be used for the NEB.

      ml2relax: boolean. True to use ML to relax the initial and final structure guesses. 
      False if initial and final structures were relaxed beforehand.
      
      intermediate_samples: int. Number of intermediate samples to be used in constructing the NEB"""
      
      self.starting_images = copy.deepcopy(starting_images)
      self.ml2relax = ml2relax
      self.intermediate_samples = intermediate_samples
      
    def run(self, calc, filename):
      """
      Runs NEB calculations.
      Parameters
      ----------
      calc: object. Calculator to be used to run method.
      filename: str. Label to save generated trajectory files."""
      
      initial = self.starting_images[0].copy()
      final = self.starting_images[-1].copy()

      if self.ml2relax:
        # Relax initial and final images
        ml_initial = initial
        ml_initial.set_calculator(calc)
        ml_final = final
        ml_final.set_calculator(calc)
        qn = BFGS(ml_initial, trajectory='initial.traj', logfile='initial_relax_log.txt')
        qn.run(fmax=0.01, steps=100)
        qn = BFGS(ml_final, trajectory='final.traj', logfile='final_relax_log.txt')
        qn.run(fmax=0.01, steps=100)
        initial = ml_initial.copy()
        final = ml_final.copy()

      initial.set_calculator(calc)
      final.set_calculator(calc)

      images = [initial]
      for i in range(self.intermediate_samples):
        image = initial.copy()
        image.set_calculator(calc) 
        images.append(image)
      images.append(final)

      print('NEB BEING BUILT')
      neb = SingleCalculatorNEB(images)
      neb.interpolate()
      print('NEB BEING OPTIMISED')
      opti = BFGS(neb, trajectory=filename+".traj", logfile='al_neb_log.txt')
      opti.run(fmax=0.01, steps=100)
      print('NEB DONE')

      """ 
      The following code is used to visualise the NEB at every iteration 

      built_neb = NEBTools(images)
      barrier, dE = built_neb.get_barrier()
      max_force = built_neb.get_fmax()
      fig = built_neb.plot_band()
      plt.show()
      """
    def get_trajectory(self, filename, start_count, end_count, interval):
      trajectory = ase.io.read(
        filename + ".traj", "{}:{}:{}".format(start_count, end_count, interval)
      )
      return trajectory

In [0]:
#https://wiki.fysik.dtu.dk/ase/tutorials/neb/diffusion.html#diffusion-tutorial
#Surface Diffusion Energy Barriers
#Building your structure

def construct_geometries(parent_calc, ml2relax):
  counter_calc = CounterCalc(parent_calc)
  # Initial structure guess
  initial_slab = fcc100('Cu', size=(2, 2, 3))
  add_adsorbate(initial_slab, 'O', 1.7, 'hollow')
  initial_slab.center(axis=2, vacuum=4.0)
  mask = [atom.tag > 1 for atom in initial_slab]
  initial_slab.set_constraint(FixAtoms(mask=mask))

  initial_slab.set_calculator(counter_calc)

  # Final structure guess
  final_slab = initial_slab.copy()
  final_slab[-1].x += final_slab.get_cell()[0, 0] / 3
  final_slab.set_calculator(counter_calc)

  if not ml2relax:
    qn = QuasiNewton(initial_slab, trajectory='initial.traj', logfile='initial_relax_log.txt')
    qn.run(fmax=0.01, steps=100)
    qn = QuasiNewton(final_slab, trajectory='final.traj', logfile='final_relax_log.txt')
    qn.run(fmax=0.01, steps=100)
    initial_slab = read("initial.traj", "-1")
    final_slab = read("final.traj", "-1")
  else:
    initial_slab = attach_sp_calc(initial_slab)
    final_slab = attach_sp_calc(final_slab)

  initial_force_calls = counter_calc.force_calls
  return initial_slab, final_slab, initial_force_calls

In [0]:
# Define symmetry functions
Gs = {}
Gs["G2_etas"] = np.logspace(np.log10(0.05), np.log10(5.0), num=4)
Gs["G2_rs_s"] = [0] * 4
Gs["G4_etas"] = [0.005]
Gs["G4_zetas"] = [1.0, 4.0]
Gs["G4_gammas"] = [+1.0, -1]
Gs["cutoff"] = 5.876798323827276  # EMT asap_cutoff: False

# Define morse parameters if Delta-ML model, o/w morse = False
morse_params = {
    "C": {"re": 0.972, "D": 6.379, "sig": 0.477},
    "O": {"re": 1.09, "D": 8.575, "sig": 0.603},
    "Cu": {"re": 2.168, "D": 3.8386, "sig": 1.696},
}

# Training parameters for the active learning framework
training_params = {
      "al_convergence": {"method": "iter", "num_iterations": 5},
      "samples_to_retrain": 5,
      "Gs": Gs,
      "morse": True,
      "morse_params": morse_params,
      "forcetraining": True,
      "cores": 1,
      "optimizer": torch.optim.LBFGS,
      "batch_size": 1000,
      "criterion": CustomMSELoss,
      "num_layers": 3,
      "num_nodes": 20,
      "force_coefficient": 0.04,
      "learning_rate": 1e-1,
      "epochs": 200,
      "test_split": 0,
      "shuffle": False,
      "verbose": 1,
      "filename": "neb_example",
      "file_dir": "./"
}

ml2relax = False
parent_calc = EMT()
initial, final, initial_force_calls = construct_geometries(parent_calc=parent_calc, ml2relax=ml2relax)
images = [initial]
images.append(final)

# Define AL scheme

learner = AtomisticActiveLearner(
    training_data=images,
    training_params=training_params,
    parent_calc=parent_calc,
    ensemble=False
)

# Learn the corresponding atomistic method
total_neb_images = 5  # N + 2 where N is the number of intermediate images and 2 is for initial and final structures

learner.learn(
    atomistic_method=NEBcalc(
        starting_images=images, 
        ml2relax=ml2relax, 
        intermediate_samples=total_neb_images-2), 
		query_strategy=random_query
	)

Visualize AL Results

In [0]:
def neb_analysis(images, label, ax=None):
  nebtools = NEBTools(images)
  barrier, dE = nebtools.get_barrier()
  max_force = nebtools.get_fmax()
  fig = nebtools.plot_band(ax=ax)
  fig.savefig(label+'.png', dpi=300)
  return barrier, dE, max_force

images = read(f'neb_example_iter_{training_params["al_convergence"]["num_iterations"]}.traj@-{total_neb_images}:')

# to verify ml_computed images...evaluate with parent calculator
emt_evaluated_images = []
for image in images:
  #image = image.copy()
  image.set_calculator(EMT())
  emt_evaluated_images.append(image)

barrier_ml, dE_ml, max_force_ml = neb_analysis(images, label="ml_diffusion_neb")

In [0]:
barrier_emt_ml, dE_emt_ml, max_force_emt_ml = neb_analysis(emt_evaluated_images, label="emt@ml_diffusion_neb")
print(f"ML barrier pred = {barrier_ml}, ml dE pred = {dE_ml}, ml max force = {max_force_ml}")
print(f"emt@ml barrier = {barrier_emt_ml}, emt@ml dE = {dE_emt_ml}, emt@ml max force = {max_force_emt_ml}")
print(f"AL: Total number of force calls = {initial_force_calls} (initial data) + {learner.parent_calls} (AL-NEB) = {initial_force_calls+learner.parent_calls}")

#### Code to generate true NEB

In [0]:
emt_counter = CounterCalc(EMT())

if not ml2relax:
  # Load previously computed relaxed states
  initial_slab = read("initial.traj", "-1")
  final_slab = read("final.traj", "-1")
else:
  initial.set_calculator(emt_counter)
  final.set_calculator(emt_counter)
  qn = QuasiNewton(initial, trajectory='parent_initial.traj', logfile='initial_relax_log.txt')
  qn.run(fmax=0.01, steps=200)
  qn = QuasiNewton(final, trajectory='parent_final.traj', logfile='final_relax_log.txt')
  qn.run(fmax=0.01, steps=200)
  initial_slab = read("parent_initial.traj", "-1")
  final_slab = read("parent_final.traj", "-1")

images = [initial]
for i in range(3):
    image = final.copy()
    image.set_calculator(emt_counter)
    images.append(image)
images.append(final)

neb = SingleCalculatorNEB(images)
neb.interpolate()
qn = BFGS(neb, trajectory='actual_neb.traj')
qn.run(fmax=0.01)

In [0]:
images = read(f'actual_neb.traj@-{total_neb_images}:')
nebtools = NEBTools(images)

Ef, dE, max_force = neb_analysis(images, label="actual_emt_neb")

print(f"Ef = {Ef}, dE = {dE}, max_force = {max_force}")
print(f"Total number of force calls = {initial_force_calls} (initial data) + {emt_counter.force_calls} Total = {initial_force_calls+emt_counter.force_calls}")